# Hello F#

This is a .net interactive notebook with some sample code of the basic of F# that you play around with.

Things we will cover in this notebook is:

* Simple types
* What is `let`
* More complex types
* Pattern matching

## Simple types

Types is one of the most important strength in F#. I've devided types into two sections, one around simple types and then one more advanced types later on.

Types are defined using the `type` keyword. That is used for all the different types you can define, records, discriminated unions, classes, abstract classes or interfaces (yes you can do classes and interfaces in F#)

Let's see some examples

In [ ]:
// Type alias 
type Age = int // This is just an alias over a native built-in type
type Name = string

// Record
type Person = {
    Name: Name
    Age: Age
}

type Weirdo = 
    | Something of Person
    | Number of int

// Discriminated union
type Employee =
    | Manager of Person
    | Employee of Person
    | Weirdo of Weirdo

## All you need is `let`

`let` is the keyword used to define and assign a variable in F#. You use it to define functions and values. One thing that is important in F# is that everything is an expression. So everything has an output.

In [ ]:
// Integer
let x = 10

// String
let y = "Hello"

// Record 
let tomas = { Name = "Tomas"; Age = 30 }
let tomas2: Person = {
    Person.Name = "Tomas" // No need for ;
    Age = 30
}
// DUs
let employee = Employee tomas
let manager = Manager tomas2
let weirdo = Weirdo (Number 10)

// Functions
let add x y = x + y
add 5 6, tomas = tomas2, employee = manager, weirdo, weirdo = employee
// add 5.5 5.6 // Fails since add has been inferred to be of type int

In [ ]:
// More functions
let changeName name person = { person with Name = name }
let ole = changeName "Ole" tomas
(ole, tomas, ole = tomas, tomas = tomas)


In [ ]:
tomas // is still unchanged

In [ ]:
let changeEmployeeName name employee = 
    match employee with
    | Manager person -> Manager (changeName name person)
    | Employee person -> Employee (changeName name person)

changeEmployeeName "ole" manager

## How do you work with "null"?

> Null References: The Billion Dollar Mistake - Tony Hoare (creator of null)

In "pure" F# null isn't allowed, so how do you work with null?

The answer is `Option`. If you want to make something optional, you must define it as an `Option` which forces you to do the right thing to get the code to compile.

In [ ]:
type Name = { FirstName: string; LastName: string; MiddleName: string option }
let name1 = { FirstName = "Tomas"; LastName = "Jansson"; MiddleName = None }
let name2 = { FirstName = "Tomas"; LastName = "Jansson"; MiddleName = Some "Doe" }

let names = ["viktor"; "tomas"; "ole"]
let viktor = names |> List.tryFind (fun n -> n = "viktor2") |> Option.defaultValue "Not found"

let printName name =
    match name.MiddleName with
    | None -> sprintf "%s %s" name.FirstName name.LastName
    | Some middleName -> sprintf "%s (%s) %s" name.FirstName middleName name.LastName

printName name1, printName name2, viktor


## Currying and partial applications

Currying and partial applications are central concept in programming in general, and you can also use it in most languages today that treats functions as first class citizens (C# does as well).

Currying is to allow a function that takes multiple arguments to be written as a sequence of functions where each function takes one argument.

    let add1 (x,y) = x + y // function that takes to arguments
    let add2 x y = x + y // curried version

Partial applications is what you got when you call a curried function with only some of its values

    let add100 = add2 100 // This will call add2 with argumenet 100 returning a new function where x is set to be 100

In [ ]:
let add1 (x,y) = x + y
let add2 x y = x + y
let add100 = add2 100

(add1(5,4), add 5 4, add100 5)

In [ ]:
let doMath operator x y = operator x y
let doAddition = doMath (fun x y -> x + y)
let doSubtraction = doMath (fun x y -> x - y)
let doMultiplication = doMath (fun x y -> x * y)
let doDivision = doMath (fun x y -> x / y)

doAddition 10 10, doSubtraction 10 4, doMultiplication 10 10, doDivision 100 10

## Piping (|>)

Piping is very commong in F#, and it is a lot like bash piping. The syntax in short is

    value |> function

this is exactly the same as calling

    function value

In [ ]:
let parse x = int x
let validate x = 
    if x > 2 then x else raise (exn "Invalid number")
let double x = x * 2

let application x =
    x |> parse |> validate |> double

"5" |> application

## More types

Discriminated unions is one of the most powerful features in F# (in my opinion). So let's explore them a little bit more.

Basic structure of discriminated unions:

```
type <Name of DU> =
    | <case> (of data)

type Option<'T> =
    | None
    | Some of 'T
```

In [ ]:
type MyOption<'T> =
    | MySome of 'T
    | MyNone

let x = MySome 5
let y = MyNone

let toString x =
    match x with
    | MySome x -> sprintf "%A" x
    | MyNone -> sprintf "None"

(toString x, toString y)

In [2]:
// More advanced DU

// Use case, create a loyalty engine! :)
// We should be able to define rules based on order line which will say if they should get bonus point or not

// Filter will define if the order line should get bonus point
type Filter<'T> =
    | Expr of ('T -> bool)
    | And of Filter<'T> list
    | Or of Filter<'T> list
    | Not of Filter<'T>
 
// Sink defins how bonus points are calculated
type Sink<'T> = 'T -> 'T

// Workflow defines how a flow with multiple filters and sinks should be created
type Workflow<'T> =
    | Sink of Sink<'T>
    | Filter of Filter<'T> * Workflow<'T>

type OrderLine = {
    Brand: string
    ArticleId: string
    AmountPerItem: int
    Quantity: int
}

type OrderHeader = {
    OrderId: string
    TotalAmount: int
}
type Order = {
    OrderHeader: OrderHeader
    OrderLines: OrderLine list
}

// In the real world this will be part of order, but simplified for the sake of this example
type OrderLineResult = {
    Brand: string
    ArticleId: string
    Amount: int
    Points: int
    Campaign: string
}
type OrderResult = {
    OrderHeader: OrderHeader
    MatchedCampaigns: string list
    BonusPoints: int
    OrderLineResults: OrderLineResult list
}

type OrderLineCalculation = {
    OrderLineResult: OrderLineResult
    OrderHeader: OrderHeader
    OrderLines: OrderLine list
}

let setCampaign campaign orderLineCalculation =
    {orderLineCalculation with OrderLineResult = { orderLineCalculation.OrderLineResult with Campaign = campaign } } 

let setPoints points orderLineCalculation =
    {orderLineCalculation with OrderLineResult = { orderLineCalculation.OrderLineResult with Points = points } } 

// Sink function that gives double points
let doublePointSink (orderLineCalculation) = 
    orderLineCalculation
    |> setPoints (orderLineCalculation.OrderLineResult.Amount * 2)
    |> setCampaign "Double points"

// Sink function that gives one point per crown
let baseSink (orderLineCalculation) =
    orderLineCalculation 
    |> setPoints (orderLineCalculation.OrderLineResult.Amount)
    |> setCampaign "Base"

let triplePointSink orderLineCalculation =
    orderLineCalculation
    |> setPoints (orderLineCalculation.OrderLineResult.Amount * 3)
    |> setCampaign "Triple points"

let quadruplePointSink orderLineCalculation =
    orderLineCalculation
    |> setPoints (orderLineCalculation.OrderLineResult.Amount * 4)
    |> setCampaign "Quadruple points"

// Filter to make sure amount for orderline is above 5000
let atLeast5000Filter = Expr (fun orderLineCalculation -> orderLineCalculation.OrderLineResult.Amount >= 5000)
// Filter orderline on brand Apple
let appleBrandFilter = Expr (fun orderLineCalculation -> orderLineCalculation.OrderLineResult.Brand = "Apple")
// Filter orderline on brand Samsung
let samsungBrandFilter = Expr (fun orderLineCalculation -> orderLineCalculation.OrderLineResult.Brand = "Samsung")
// Filter orderline on total amount of order is above or equal to 20000
let totalAmountFilter = Expr (fun orderLineCalculation -> orderLineCalculation.OrderHeader.TotalAmount >= 20000)
// Filter orderline on article id "SA"
let articleQuadrupleFilter = Expr (fun orderLineCalculation -> orderLineCalculation.OrderLineResult.ArticleId = "SA")
// Combine the two brand filters so it can be either one of them
let appleOrSamsungFilter = Or [appleBrandFilter; samsungBrandFilter]

// Create workflow that matches all order lines
let basicWorkflow = Sink baseSink
// Create workflow that matches order lines with amount above 5000 and it matches the brand filter, apply double points
let doublePointWorkflow = Filter (atLeast5000Filter, (Filter (appleOrSamsungFilter, (Sink doublePointSink))))
// Give all order lines at least triple points on orders with total amount above 20000
let triplePointFlow = Filter (totalAmountFilter, (Sink triplePointSink))
// Give order lines of article id SA quadruple points
let quadrupleFlow = Filter(articleQuadrupleFilter, (Sink quadruplePointSink))

let toOrderLineCalculation orderHeader orderLines (orderLine: OrderLine) = 
    {
        OrderLineResult = {
            Brand = orderLine.Brand
            ArticleId = orderLine.ArticleId
            Amount = orderLine.AmountPerItem * orderLine.Quantity
            Points = 0
            Campaign = ""
        }
        OrderHeader = orderHeader
        OrderLines = orderLines
    }

let rec execFilter filter orderLine =
    match filter with
    | Expr func -> func orderLine
    | And funcs -> funcs |> List.fold (fun state func -> state && (execFilter func orderLine)) true
    | Or funcs -> funcs |> List.exists (fun func -> execFilter func orderLine)
    | Not func -> not (execFilter func orderLine)

let rec execute workflow orderLine =
    match workflow with
    | Sink sink -> sink orderLine
    | Filter (filter, nextStep) ->
        if execFilter filter orderLine 
        then execute nextStep orderLine
        else orderLine

let executeMany workflows orderLine =
    workflows
    // Do the calculation for all workflows
    |> List.map (fun workflow -> execute workflow orderLine)
    // Pick the result with the most points
    |> List.maxBy (fun orderLineCalculation -> orderLineCalculation.OrderLineResult.Points)

let processOrder workflows (order: Order) =
    let orderLineCalculation = 
        order.OrderLines
        |> List.map (fun orderLine -> toOrderLineCalculation order.OrderHeader order.OrderLines orderLine)
        |> List.map (executeMany workflows)
    {
        MatchedCampaigns = orderLineCalculation |> List.map (fun olc -> olc.OrderLineResult.Campaign) |> List.distinct
        OrderHeader = order.OrderHeader
        BonusPoints = orderLineCalculation |> List.sumBy(fun olc -> olc.OrderLineResult.Points)
        OrderLineResults = (orderLineCalculation |> List.map (fun olc -> olc.OrderLineResult))
    }

let appleOrderBelow5000 = { Brand = "Apple"; ArticleId = "AA"; AmountPerItem = 1000; Quantity = 1 }
let appleOrderAbove5000 = { Brand = "Apple"; ArticleId = "AB"; AmountPerItem = 5001; Quantity = 1 }
let samsungOrderBelow5000 = { Brand = "Samsung"; ArticleId = "SA"; AmountPerItem = 1000; Quantity = 1 }
let samsungOrderAbove5000 = { Brand = "Samsung"; ArticleId = "SB"; AmountPerItem = 5001; Quantity = 1 }
let order1 = {
    OrderHeader = { OrderId = "1"; TotalAmount = 12002 }
    OrderLines = [appleOrderBelow5000; appleOrderAbove5000; samsungOrderBelow5000; samsungOrderAbove5000]
}
let order2 = {
    OrderHeader = { OrderId = "1"; TotalAmount = 22002 }
    OrderLines = [appleOrderBelow5000; appleOrderAbove5000; samsungOrderBelow5000; samsungOrderAbove5000]
}

[order1; order2] |> List.map (processOrder [doublePointWorkflow; basicWorkflow; triplePointFlow; quadrupleFlow])
// order.OrderLines
// |> List.map (fun orderLine -> toOrderLineCalculation order.OrderHeader order.OrderLines orderLine)
// |> List.map (execute workflow)


index OrderHeader MatchedCampaigns BonusPoints OrderLineResults 0 OrderId TotalAmount 1 12002 [ Base, Double points, Quadruple points ] 25004 FSharpList<OrderLineResult>
 - Brand: Apple
 ArticleId: AA
 Amount: 1000
 Points: 1000
 Campaign: Base
 - Brand: Apple
 ArticleId: AB
 Amount: 5001
 Points: 10002
 Campaign: Double points
 - Brand: Samsung
 ArticleId: SA
 Amount: 1000
 Points: 4000
 Campaign: Quadruple points
 - Brand: Samsung
 ArticleId: SB
 Amount: 5001
 Points: 10002
 Campaign: Double points 1 OrderId TotalAmount 1 22002 [ Triple points, Quadruple points ] 37006 FSharpList<OrderLineResult>
 - Brand: Apple
 ArticleId: AA
 Amount: 1000
 Points: 3000
 Campaign: Triple points
 - Brand: Apple
 ArticleId: AB
 Amount: 5001
 Points: 15003
 Campaign: Triple points
 - Brand: Samsung
 ArticleId: SA
 Amount: 1000
 Points: 4000
 Campaign: Quadruple points
 - Brand: Samsung
 ArticleId: SB
 Amount: 5001
 Points: 15003
 Campaign: Triple points